In [1]:
import pandas as pd
import numpy as np
from itertools import chain

import re

import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

import csv
from io import StringIO

from envparse import env
env.read_envfile("..//.env")

## ETL (выделение сущностей meta)

In [2]:
data = []
tmp = []

with open("../DATA/all_meta.dat", "r") as f:
    for i in f:
        if i=='*$*\n':
            data.append(" ".join(tmp))
            tmp = []
        tmp.append(i)

data = data[1:]

In [3]:
#Частотный анализ

def counter(list_element):
    """Счетчик повторений элементов последовательности"""
    count = {}
    for element in list_element:
        if count.get(element, None):
            count[element] += 1
        else:
            count[element] = 1

    # сортируем словарь по количеству повторений слов в тексте
    sorted_values = sorted(count.items(), key=lambda tpl: tpl[1], reverse=True)
    return dict(sorted_values)

all_data = [i.split(" ") for i in data]
all_data = list(chain.from_iterable(all_data))

#результат
# with open("..\\freq_analize.txt", "w") as f:
#     f.write(str(counter(all_data)))

In [4]:
# Заявитель - applicant
# Поставщики - кандидаты, сумма - candidates
# Поставщик - победитель, сумма
# Дата публикации заявки
# номер торгов
# место/адрес
# телефон
# почта
# наименование проекта

#ищем все сущности которые заканчивваются на ":"
entity_key =[
              "Контингент:",
              "Участник:",
              "Заявители:",
              "Покупатель:",
              "Организатор:",
                "Тел.:",
                "почта:",
                "Информация:",
                "Дата выхода:",
                "Агент по торгам:",
                "Информация о кандидатах:",
                "Контактное лицо:",
                "Цены:",
                "номер тендера:",
                "Номер проекта:",
                "Адрес:",
                "Производитель:",
                "Слово:",
                "(руководитель проекта):",
                "агент:",
            ]


topic = []
res_1=[]
res_2=[]
res_3=[]
res_4=[]
res_5=[]
res_6=[]
res_7=[]
res_8=[]
res_9=[]
res_10=[]
res_11=[]
res_12=[]
res_13=[]
res_14=[]
res_15=[]
res_16=[]
res_17=[]
res_18=[]
res_19=[]
res_20=[]


for i in data:
    tmp =[]
    tmp=[m.start() for m in re.finditer('\n',i)][1:3]
    topic.append(i[tmp[0]+2:tmp[1]])



def find_substring_by_keyword(input_string, keyword, end_char):
    pattern = re.compile(f'{re.escape(keyword)}(.*?){re.escape(end_char)}')
    match = pattern.search(input_string)
    
    if match:
        return match.group(1)
    else:
        return None



for i in data:
    res_1.append(find_substring_by_keyword(i,entity_key[0],"\n"))
    res_2.append(find_substring_by_keyword(i,entity_key[1],"\n"))
    res_3.append(find_substring_by_keyword(i,entity_key[2],"\n"))
    res_4.append(find_substring_by_keyword(i,entity_key[3],"\n"))
    res_5.append(find_substring_by_keyword(i,entity_key[4],"\n"))
    res_6.append(find_substring_by_keyword(i,entity_key[5],"\n"))
    res_7.append(find_substring_by_keyword(i,entity_key[6],"\n"))
    res_8.append(find_substring_by_keyword(i,entity_key[7],"\n"))
    res_9.append(find_substring_by_keyword(i,entity_key[8],"\n"))
    res_10.append(find_substring_by_keyword(i,entity_key[9],"\n"))
    res_11.append(find_substring_by_keyword(i,entity_key[10],"\n"))
    res_12.append(find_substring_by_keyword(i,entity_key[11],"\n"))
    res_13.append(find_substring_by_keyword(i,entity_key[12],"\n"))
    res_14.append(find_substring_by_keyword(i,entity_key[13],"\n"))
    res_15.append(find_substring_by_keyword(i,entity_key[14],"\n"))
    res_16.append(find_substring_by_keyword(i,entity_key[15],"\n"))
    res_17.append(find_substring_by_keyword(i,entity_key[16],"\n"))
    res_18.append(find_substring_by_keyword(i,entity_key[17],"\n"))
    res_19.append(find_substring_by_keyword(i,entity_key[18],"\n"))
    res_20.append(find_substring_by_keyword(i,entity_key[19],"\n"))
    
    
    
 
def clear_(in_lst):
    return [i.replace("&quot;","").replace("&gt;","").replace("&lt;","").replace("&apos;","").strip().replace("  "," ") if not(pd.isna(i)) else np.nan for i in in_lst ]
        
topic = clear_(topic)
res_1 = clear_(res_1)
res_1 = [i.replace(")","") if not(pd.isna(i)) else np.nan for i in res_1]
res_2 = clear_(res_2)
res_3 = clear_(res_3)
res_4 = clear_(res_4)
res_5 = clear_(res_5)
res_6 = clear_(res_6)
res_7 = clear_(res_7)
res_8 = clear_(res_8)
res_9 = clear_(res_9)
res_10 = clear_(res_10)
res_11 = clear_(res_11)
res_12 = clear_(res_12)
res_13 = clear_(res_13)
res_14 = clear_(res_14)
res_15 = clear_(res_15)
res_16 = clear_(res_16)
res_17 = clear_(res_17)
res_18 = clear_(res_18)
res_19 = clear_(res_19)
res_20 = clear_(res_20)
    


df = pd.DataFrame({entity_key[0]:res_1,
                   "topic":topic,
              entity_key[1]:res_2,
              entity_key[2]:res_3,
              entity_key[3]:res_4,
              entity_key[4]:res_5,
              entity_key[5]:res_6,
              entity_key[6]:res_7,
              entity_key[7]:res_8,
              entity_key[8]:res_9,
              entity_key[9]:res_10,
              entity_key[10]:res_11,
              entity_key[11]:res_12,
              entity_key[12]:res_13,
              entity_key[13]:res_14,
              entity_key[14]:res_15,
              entity_key[15]:res_16,
              entity_key[16]:res_17,
              entity_key[17]:res_18,
              entity_key[18]:res_19,
              entity_key[19]:res_20,
              })

df[:3]
# df.to_excel("result.xlsx", index=False)


,Контингент:,topic,Участник:,Заявители:,Покупатель:,Организатор:,Тел.:,почта:,Информация:,Дата выхода:,...,Информация о кандидатах:,Контактное лицо:,Цены:,номер тендера:,Номер проекта:,Адрес:,Производитель:,Слово:,(руководитель проекта):,агент:
0,CGN-20180529004,Установлены результаты первого из трех башен в...,NaN,Чайна инжиниринг лимитед,NaN,NaN,NaN,luozhenggang@cgnpc.com.cn,NaN,2023-04-11,...,,NaN,RMB14 068 380.00,NaN,NaN,NaN,NaN,NaN,(подпись),(глава)
1,CGN-20180529005,Информация о результатах в системе физической ...,NaN,Чайна инжиниринг лимитед,NaN,NaN,NaN,zhangtong@cgnpc.com.cn,NaN,2023-04-10,...,,NaN,NaN,NaN,NaN,NaN,Lake South Huawei Electronics Inc.,NaN,(подпись),(глава)
2,NaN,"3-я атомная атомная электростанция Китая, Авст...",NaN,Синьхуа,China Construction Corp.,NaN,0755-84437846,CMECCD@cgnpc.com.cn,NaN,2023-04-17,...,,NaN,RMB: 674 850 653.49,NaN,NaN,NaN,NaN,NaN,(подпись),(глава)


## Подключение к PG

In [42]:
pg_host = env("PG_HOST")
pg_port = env("PG_PORT")
pg_db = env("PG_DATABASE")
pg_user = env("PG_USER")
pg_psw = env("PG_PASSWORD")


alchemyEngine           = create_engine(f'postgresql+psycopg2://{pg_user}:{pg_psw}@{pg_host}:{pg_port}/{pg_db}',
                                        pool_recycle=3600)
postgreSQLConnection    = alchemyEngine.connect()
postgreSQLConnection.autocommit = True

def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name
        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)


In [43]:
postgreSQLConnection.rollback()

In [44]:
# Очистка и создание БД
f = open("../DDL/create_db.sql", "r").read()
postgreSQLConnection.execute(text(f))
postgreSQLConnection.commit()

In [45]:
#Заявители
tmp = (
        df[["Заявители:","Тел.:","почта:",]]
       .drop_duplicates(subset="Заявители:", keep='first')
       .dropna(subset=['Заявители:'])
       .reset_index(drop=True)
       .reset_index()
       .rename(columns={"index":"id",
                        "Заявители:":"name",
                        "Тел.:":"phone",
                        "почта:":"email",
                        })
       )

companies_df = pd.read_sql("select * from public.companies c ", postgreSQLConnection)
companies_df = pd.concat([tmp,companies_df])

companies_df.to_sql('companies', 
                    postgreSQLConnection, 
                    method=psql_insert_copy,
                    if_exists='replace',
                    index=False, 
                    chunksize=1000)
postgreSQLConnection.commit()

In [46]:
#Тендеры
tmp = (
        df[["Контингент:",
            "topic",
            "Дата выхода:",
            "Цены:",]]
       .drop_duplicates(subset="Контингент:", keep='first')
       .dropna(subset=['Контингент:'])
       .reset_index(drop=True)
       .reset_index()
       .rename(columns={"index":"id",
                        "Контингент:":"cgn",
                        "Дата выхода:":"createdat",
                        "Цены:":"price"
                        })
       )

geo_df = pd.read_csv("../DATA/geo_data.csv")
geo_df.CGN = geo_df.CGN.astype('string')
geo_df.dropna(subset=["CGN"],inplace=True)
geo_df.CGN = geo_df.CGN.apply(lambda x: "CGN-" + x.replace(".0",""))
geo_df.rename(columns={"CGN":"cgn"}, inplace=True)
geo_df=geo_df.drop(columns='index')

tenders_df = pd.read_sql("select * from public.tenders c ", postgreSQLConnection)
tenders_df = pd.concat([tmp,tenders_df]).drop(columns=["location","organization"])
tenders_df = pd.merge(tenders_df,
                      geo_df,
                      left_on='cgn',
                      right_on='cgn',
                      how='left')
tenders_df.createdat = pd.to_datetime(tenders_df.createdat)


tenders_df.to_sql('tenders', 
                    postgreSQLConnection, 
                    method=psql_insert_copy,
                    if_exists='replace',
                    index=False, 
                    chunksize=1000)
postgreSQLConnection.commit()

In [47]:
#Связка Заявитель-Тендер
tmp = pd.merge(df[["Контингент:","Заявители:"]],
                tenders_df[["cgn","id"]],
                left_on = 'Контингент:',
                right_on= 'cgn',
                how='left').rename(columns={'id':'id_tender'})
tmp = pd.merge(tmp,
               companies_df[["name","id"]],
                left_on = 'Заявители:',
                right_on= 'name',
                how='left').rename(columns={'id':'id_company'})
tmp = tmp.dropna(subset='cgn').drop_duplicates(subset='cgn').reset_index(drop=True).reset_index()
tmp["is_main"] = True

tmp = tmp[["index","id_tender","id_company","is_main"]].rename(columns={"index":"id"})

link_tender_company_df = pd.read_sql("select * from public.tenders_companies c ", postgreSQLConnection)
link_tender_company_df = pd.concat([tmp,link_tender_company_df])

link_tender_company_df.to_sql('tenders_companies', 
                    postgreSQLConnection, 
                    method=psql_insert_copy,
                    if_exists='replace',
                    index=False, 
                    chunksize=1000)
postgreSQLConnection.commit()

In [48]:
#Справочник работ
tmp = pd.read_csv("../DATA/material.csv", sep=';')
tmp.reset_index(inplace=True)
tmp.rename(columns={"index":"id",
                    "tenderid":"tenderId",
                    "workid":"title",
                    "type":"workType"}, inplace=True)
tmp = tmp[["title","workType"]].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={"index":"id"})

work_df = pd.read_sql("select * from public.works c ", postgreSQLConnection)
work_df = pd.concat([tmp,work_df])

work_df.to_sql('works', 
                    postgreSQLConnection, 
                    method=psql_insert_copy,
                    if_exists='replace',
                    index=False, 
                    chunksize=1000)
postgreSQLConnection.commit()


In [49]:
#Справочник работы-тендеры
tmp = pd.read_csv("../DATA/material.csv", sep=';')
tmp.reset_index(inplace=True)
tmp.rename(columns={"index":"id",
                    "tenderid":"tenderId",
                    "workid":"title",
                    "type":"workType"}, inplace=True)
tmp.tenderId = tmp.tenderId.apply(lambda x: "CGN-"+ x if not(pd.isna(x)) else np.nan)
tmp = pd.merge(tmp[["tenderId","title"]],
               tenders_df[["id","cgn"]],
               left_on="tenderId",
               right_on="cgn",
               how="left").drop(columns="tenderId").rename(columns={"id":"id_tender"})
tmp = pd.merge(tmp,
               work_df,
               left_on="title",
               right_on="title",
               how="left").rename(columns={"id":"id_work"})
tmp = tmp[["id_tender","id_work"]].reset_index(drop=True).reset_index().rename(columns={"index":"id"})

link_tender_work_df = pd.read_sql("select * from public.tenders_works c ", postgreSQLConnection)
link_tender_work_df = pd.concat([tmp,link_tender_work_df]).dropna(subset=["id_tender","id_work"])
link_tender_work_df

link_tender_work_df.to_sql('tenders_works', 
                    postgreSQLConnection, 
                    method=psql_insert_copy,
                    if_exists='replace',
                    index=False, 
                    chunksize=1000)
postgreSQLConnection.commit()
